<a href="https://colab.research.google.com/github/YuriHermont1995/TesouroDireto-PythonPandas/blob/main/API_TD_HML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **API Tesouro Direto**
## **Notebook para Homologação de funcionalidades sobre os dados coletados no TD** 

## **Consulta TD Historico de taxas dos titulos**

### **Buscando Dados**


In [2]:
import pandas as pd
aux = pd.read_csv('https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv', sep=';')

In [3]:
aux.head()

,Tipo Titulo,Data Vencimento,Data Base,Taxa Compra Manha,Taxa Venda Manha,PU Compra Manha,PU Venda Manha,PU Base Manha
0,Tesouro Prefixado com Juros Semestrais,01/01/2029,09/10/2019,"6,92","7,04","1228,86","1220,07","1219,74"
1,Tesouro Prefixado com Juros Semestrais,01/01/2029,10/10/2019,"6,85","6,97","1234,35","1225,51","1225,19"
2,Tesouro Prefixado com Juros Semestrais,01/01/2029,11/10/2019,"6,77","6,89","1240,61","1231,72","1231,39"
3,Tesouro Prefixado com Juros Semestrais,01/01/2029,14/10/2019,"6,63","6,75","1251,41","1242,42","1242,10"
4,Tesouro Prefixado com Juros Semestrais,01/01/2029,15/10/2019,"6,62","6,74","1252,49","1243,49","1243,17"


In [4]:
aux.dtypes

Tipo Titulo          object
Data Vencimento      object
Data Base            object
Taxa Compra Manha    object
Taxa Venda Manha     object
PU Compra Manha      object
PU Venda Manha       object
PU Base Manha        object
dtype: object

In [5]:
tamanhoInicial = aux.shape[0]
tamanhoInicial

116800

**reflexão**

In [6]:
print('Foram buscadas respectivamente linhas e colunas igual a ',aux.shape)
print('na base de dados do historico do TesouroDireto')

Foram buscadas respectivamente linhas e colunas igual a  (116800, 8)
na base de dados do historico do TesouroDireto


### **Tratando nomes de coluna, retirada de espaços e outras necessidades**

In [7]:
aux.columns

Index(['Tipo Titulo', 'Data Vencimento', 'Data Base', 'Taxa Compra Manha',
       'Taxa Venda Manha', 'PU Compra Manha', 'PU Venda Manha',
       'PU Base Manha'],
      dtype='object')

In [8]:
def funcLimpaColunaDataFrame(df):
  str_columns = df.columns
  renomear={}
  for value in str_columns:
    renomear[value] = value.replace('.','').replace(' ','_',2).replace(' ','').lower(  )
  df = pd.DataFrame(df.rename(columns = renomear))
  return df

In [9]:
df = funcLimpaColunaDataFrame(aux)
df.head()

,tipo_titulo,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
0,Tesouro Prefixado com Juros Semestrais,01/01/2029,09/10/2019,"6,92","7,04","1228,86","1220,07","1219,74"
1,Tesouro Prefixado com Juros Semestrais,01/01/2029,10/10/2019,"6,85","6,97","1234,35","1225,51","1225,19"
2,Tesouro Prefixado com Juros Semestrais,01/01/2029,11/10/2019,"6,77","6,89","1240,61","1231,72","1231,39"
3,Tesouro Prefixado com Juros Semestrais,01/01/2029,14/10/2019,"6,63","6,75","1251,41","1242,42","1242,10"
4,Tesouro Prefixado com Juros Semestrais,01/01/2029,15/10/2019,"6,62","6,74","1252,49","1243,49","1243,17"


In [10]:
df.tipo_titulo.unique()

array(['Tesouro Prefixado com Juros Semestrais', 'Tesouro IPCA+',
       'Tesouro Prefixado', 'Tesouro Selic',
       'Tesouro IGPM+ com Juros Semestrais',
       'Tesouro IPCA+ com Juros Semestrais'], dtype=object)

In [11]:
def funcLimpaDadosDataFrame(df):
  tipo_titulo = df.tipo_titulo
  df.tipo_titulo = df.tipo_titulo.str.replace('Tesouro','').str.replace(' ','')
  df.data_vencimento = pd.to_datetime(df.data_vencimento)
  df.data_base = pd.to_datetime(df.data_base)
  
  df.taxa_compra_manha = df.taxa_compra_manha.str.replace(',','.')
  df.taxa_compra_manha = pd.to_numeric(df.taxa_compra_manha)
  
  df.taxa_venda_manha = df.taxa_venda_manha.str.replace(',','.')
  df.taxa_venda_manha = pd.to_numeric(df.taxa_venda_manha)
  
  df.pu_compra_manha = df.pu_compra_manha.str.replace(',','.')
  df.pu_compra_manha = pd.to_numeric(df.pu_compra_manha)
  
  df.pu_venda_manha = df.pu_venda_manha.str.replace(',','.')
  df.pu_venda_manha = pd.to_numeric(df.pu_venda_manha)
  
  df.pu_base_manha = df.pu_base_manha.str.replace(',','.')
  df.pu_base_manha = pd.to_numeric(df.pu_base_manha)

  return pd.DataFrame(df)

In [12]:
df = funcLimpaDadosDataFrame(df)

In [13]:
df

,tipo_titulo,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
0,PrefixadocomJurosSemestrais,2029-01-01,2019-09-10,6.92,7.04,1228.86,1220.07,1219.74
1,PrefixadocomJurosSemestrais,2029-01-01,2019-10-10,6.85,6.97,1234.35,1225.51,1225.19
2,PrefixadocomJurosSemestrais,2029-01-01,2019-11-10,6.77,6.89,1240.61,1231.72,1231.39
3,PrefixadocomJurosSemestrais,2029-01-01,2019-10-14,6.63,6.75,1251.41,1242.42,1242.10
4,PrefixadocomJurosSemestrais,2029-01-01,2019-10-15,6.62,6.74,1252.49,1243.49,1243.17
...,...,...,...,...,...,...,...,...
116795,Prefixado,2026-01-01,2020-04-29,7.15,7.27,676.15,671.87,671.68
116796,Prefixado,2026-01-01,2020-04-30,7.09,7.21,678.48,674.19,674.01
116797,Prefixado,2026-01-01,2020-04-05,7.24,7.36,673.31,669.06,668.87
116798,Prefixado,2026-01-01,2020-05-05,6.91,7.03,685.34,681.00,680.82


**reflexão**

In [14]:
print('Agora temos tipos indentificados e nossa estrutura de dados não possui mais espaçoes')
print(' ')
df.dtypes

Agora temos tipos indentificados e nossa estrutura de dados não possui mais espaçoes
 


tipo_titulo                  object
data_vencimento      datetime64[ns]
data_base            datetime64[ns]
taxa_compra_manha           float64
taxa_venda_manha            float64
pu_compra_manha             float64
pu_venda_manha              float64
pu_base_manha               float64
dtype: object

### Limpando linhas de TD  com vencimento passado

In [15]:
vencimentos = df.data_vencimento.unique()
vencimentos

array(['2029-01-01T00:00:00.000000000', '2045-05-15T00:00:00.000000000',
       '2022-01-01T00:00:00.000000000', '2021-01-01T00:00:00.000000000',
       '2025-01-01T00:00:00.000000000', '2008-01-01T00:00:00.000000000',
       '2007-01-01T00:00:00.000000000', '2007-01-04T00:00:00.000000000',
       '2007-01-07T00:00:00.000000000', '2007-01-10T00:00:00.000000000',
       '2019-05-15T00:00:00.000000000', '2018-01-01T00:00:00.000000000',
       '2027-01-01T00:00:00.000000000', '2023-01-03T00:00:00.000000000',
       '2023-01-01T00:00:00.000000000', '2010-01-07T00:00:00.000000000',
       '2008-01-04T00:00:00.000000000', '2011-01-03T00:00:00.000000000',
       '2010-01-01T00:00:00.000000000', '2011-01-01T00:00:00.000000000',
       '2009-03-18T00:00:00.000000000', '2010-03-17T00:00:00.000000000',
       '2006-01-04T00:00:00.000000000', '2005-02-16T00:00:00.000000000',
       '2006-01-18T00:00:00.000000000', '2011-03-16T00:00:00.000000000',
       '2006-01-07T00:00:00.000000000', '2007-01-17

In [16]:
today = pd.to_datetime("today")
today

Timestamp('2021-07-22 12:45:34.897510')

In [17]:
filtro_datas = df[df['data_vencimento']<today].index
filtro_datas

Int64Index([  1319,   1320,   1321,   1322,   1323,   1324,   1325,   1326,
              1327,   1328,
            ...
            113644, 113645, 113646, 113647, 113648, 113649, 113650, 113651,
            113652, 113653],
           dtype='int64', length=75620)

In [18]:
df = df.drop(filtro_datas)
df

,tipo_titulo,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
0,PrefixadocomJurosSemestrais,2029-01-01,2019-09-10,6.92,7.04,1228.86,1220.07,1219.74
1,PrefixadocomJurosSemestrais,2029-01-01,2019-10-10,6.85,6.97,1234.35,1225.51,1225.19
2,PrefixadocomJurosSemestrais,2029-01-01,2019-11-10,6.77,6.89,1240.61,1231.72,1231.39
3,PrefixadocomJurosSemestrais,2029-01-01,2019-10-14,6.63,6.75,1251.41,1242.42,1242.10
4,PrefixadocomJurosSemestrais,2029-01-01,2019-10-15,6.62,6.74,1252.49,1243.49,1243.17
...,...,...,...,...,...,...,...,...
116795,Prefixado,2026-01-01,2020-04-29,7.15,7.27,676.15,671.87,671.68
116796,Prefixado,2026-01-01,2020-04-30,7.09,7.21,678.48,674.19,674.01
116797,Prefixado,2026-01-01,2020-04-05,7.24,7.36,673.31,669.06,668.87
116798,Prefixado,2026-01-01,2020-05-05,6.91,7.03,685.34,681.00,680.82


**reflexão**

In [19]:
print('A Manipulação de datas e da nossa estrutura de dado foi otima')
print('Agora temos apenas dados com relevancia atual, com respectivamente linhas e colunas igual a', df.shape)
print('Isso diminuiu o volume de dados para apenas ',round((df.shape[0]/tamanhoInicial)*100),'% do total inicial')

A Manipulação de datas e da nossa estrutura de dado foi otima
Agora temos apenas dados com relevancia atual, com respectivamente linhas e colunas igual a (41180, 8)
Isso diminuiu o volume de dados para apenas  35 % do total inicial


### Iniciando separação dos tipos de tesouro

Separando por tipo de taxa - Temos SELIC, IPCA+ E Prefixados


In [20]:
tipo_titulo = df.tipo_titulo.unique()
tipo_titulo

array(['PrefixadocomJurosSemestrais', 'IPCA+', 'Prefixado', 'Selic',
       'IGPM+comJurosSemestrais', 'IPCA+comJurosSemestrais'], dtype=object)

In [21]:
df_prejuros = df[df['tipo_titulo']=='PrefixadocomJurosSemestrais']
df_pre =  df[df['tipo_titulo']=='Prefixado']

df_ipcajuros = df[df['tipo_titulo']=='IPCA+comJurosSemestrais']
df_ipca =  df[df['tipo_titulo']=='IPCA+']

df_selic =  df[df['tipo_titulo']=='Selic']

In [22]:
df_selic.head()

,tipo_titulo,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
8311,Selic,2023-01-03,2018-08-06,0.02,0.06,9536.77,9518.83,9516.43
8312,Selic,2023-01-03,2018-11-06,0.02,0.06,9539.13,9521.20,9518.79
8313,Selic,2023-01-03,2018-12-06,0.02,0.06,9541.49,9523.56,9521.16
8314,Selic,2023-01-03,2018-06-13,0.02,0.06,9543.84,9525.93,9523.53
8315,Selic,2023-01-03,2018-06-14,0.02,0.06,9546.20,9528.30,9525.89


In [23]:
df_ipca.dtypes

tipo_titulo                  object
data_vencimento      datetime64[ns]
data_base            datetime64[ns]
taxa_compra_manha           float64
taxa_venda_manha            float64
pu_compra_manha             float64
pu_venda_manha              float64
pu_base_manha               float64
dtype: object

In [24]:
def funcLimpaColunaTipo(df):
  df.drop(columns='tipo_titulo',inplace=True)
  return pd.DataFrame(df)


In [25]:
df_prejuros = funcLimpaColunaTipo(df_prejuros)
df_pre = funcLimpaColunaTipo(df_pre)

df_ipcajuros = funcLimpaColunaTipo(df_ipcajuros)
df_ipca = funcLimpaColunaTipo(df_ipca)

df_selic = funcLimpaColunaTipo(df_selic)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
df_ipca

,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
174,2045-05-15,2021-03-02,3.57,3.69,1476.76,1435.96,1435.62
175,2045-05-15,2021-04-02,3.51,3.63,1497.96,1456.56,1456.21
176,2045-05-15,2021-05-02,3.55,3.67,1484.66,1443.65,1443.02
177,2045-05-15,2021-08-02,3.55,3.67,1485.01,1444.00,1443.65
178,2045-05-15,2021-09-02,3.60,3.72,1467.51,1427.01,1426.68
...,...,...,...,...,...,...,...
116561,2026-08-15,2021-07-15,3.74,3.86,2966.73,2949.38,2948.19
116562,2026-08-15,2021-07-16,3.65,3.77,2982.49,2965.05,2962.38
116563,2026-08-15,2021-07-19,3.66,3.78,2982.21,2964.78,2963.60
116564,2026-08-15,2021-07-20,3.63,3.75,2987.76,2970.30,2969.12


2º etapa de separecao dos titulos do TD - temos diferentes titulos, cada um com seu vencimento

In [27]:
tipos_ipca = df_ipca.data_vencimento.unique()
tipos_ipca

array(['2045-05-15T00:00:00.000000000', '2024-08-15T00:00:00.000000000',
       '2035-05-15T00:00:00.000000000', '2026-08-15T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [28]:
df_ipca

,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
174,2045-05-15,2021-03-02,3.57,3.69,1476.76,1435.96,1435.62
175,2045-05-15,2021-04-02,3.51,3.63,1497.96,1456.56,1456.21
176,2045-05-15,2021-05-02,3.55,3.67,1484.66,1443.65,1443.02
177,2045-05-15,2021-08-02,3.55,3.67,1485.01,1444.00,1443.65
178,2045-05-15,2021-09-02,3.60,3.72,1467.51,1427.01,1426.68
...,...,...,...,...,...,...,...
116561,2026-08-15,2021-07-15,3.74,3.86,2966.73,2949.38,2948.19
116562,2026-08-15,2021-07-16,3.65,3.77,2982.49,2965.05,2962.38
116563,2026-08-15,2021-07-19,3.66,3.78,2982.21,2964.78,2963.60
116564,2026-08-15,2021-07-20,3.63,3.75,2987.76,2970.30,2969.12


In [29]:
df_ipca24 = df_ipca[df_ipca['data_vencimento']=='2024-08-15']
df_ipca26 = df_ipca[df_ipca['data_vencimento']=='2026-08-15']
df_ipca35 = df_ipca[df_ipca['data_vencimento']=='2035-05-15']
df_ipca45 = df_ipca[df_ipca['data_vencimento']=='2045-05-15']

In [30]:
df_ipca24.head()

,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
70458,2024-08-15,2014-02-07,6.02,6.10,1357.37,1347.09,1346.60
70459,2024-08-15,2014-03-07,6.08,6.16,1350.97,1340.75,1339.43
70460,2024-08-15,2014-04-07,6.05,6.13,1354.82,1344.57,1343.74
70461,2024-08-15,2014-07-07,6.04,6.12,1357.57,1347.31,1345.85
70462,2024-08-15,2014-08-07,6.02,6.10,1360.38,1350.09,1349.10


In [31]:
df_ipca26.describe()

,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
count,358.000000,358.000000,358.000000,358.000000,358.000000
mean,2.998436,3.118436,2863.251201,2844.085084,2843.130503
std,0.526036,0.526036,105.799390,106.129646,105.750503
min,2.040000,2.160000,2514.730000,2496.380000,2495.850000
25%,2.580000,2.700000,2810.397500,2790.175000,2789.382500
50%,2.880000,3.000000,2884.570000,2864.480000,2863.855000
75%,3.490000,3.610000,2947.060000,2928.762500,2927.522500
max,4.410000,4.530000,3014.330000,2994.600000,2993.150000


In [32]:
df_pre.data_vencimento.unique()

array(['2022-01-01T00:00:00.000000000', '2023-01-01T00:00:00.000000000',
       '2026-01-01T00:00:00.000000000', '2024-01-07T00:00:00.000000000',
       '2025-01-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [33]:
df_pre22 = df_pre[df_pre['data_vencimento'] == '2022-01-01']
df_pre23 = df_pre[df_pre['data_vencimento'] == '2023-01-01']
df_pre24 = df_pre[df_pre['data_vencimento'] == '2024-01-07']
df_pre25 = df_pre[df_pre['data_vencimento'] == '2025-01-01']
df_pre26 = df_pre[df_pre['data_vencimento'] == '2026-01-01']

In [34]:
df_pre26.describe()

,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
count,358.000000,358.000000,358.000000,358.000000,358.000000
mean,7.310866,7.430866,696.777626,692.765251,692.566480
std,0.900522,0.900522,21.008727,20.858966,20.867474
min,5.810000,5.930000,613.180000,609.290000,609.080000
25%,6.530000,6.650000,685.972500,682.370000,682.155000
50%,7.190000,7.310000,695.615000,691.595000,691.410000
75%,8.167500,8.287500,712.710000,708.470000,708.297500
max,8.980000,9.100000,738.340000,734.200000,734.020000


In [35]:
df_pre24.head()

,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
113765,2024-01-07,2021-03-17,7.53,7.65,788.68,785.81,785.58
113766,2024-01-07,2021-03-18,7.56,7.68,788.19,785.32,785.09
113767,2024-01-07,2021-03-19,7.54,7.66,788.89,786.03,785.80
113768,2024-01-07,2021-03-22,7.79,7.91,783.17,780.34,780.11
113769,2024-01-07,2021-03-23,7.90,8.02,780.81,777.99,777.75


In [36]:
df_pre26

,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
113654,2026-01-01,2020-07-05,6.92,7.04,685.34,681.01,680.83
113655,2026-01-01,2020-08-05,7.09,7.21,679.40,675.12,674.94
113656,2026-01-01,2020-11-05,7.14,7.26,677.80,673.54,673.35
113657,2026-01-01,2020-12-05,6.93,7.05,685.52,681.21,681.02
113658,2026-01-01,2020-05-13,7.23,7.35,674.97,670.74,670.55
...,...,...,...,...,...,...,...
116795,2026-01-01,2020-04-29,7.15,7.27,676.15,671.87,671.68
116796,2026-01-01,2020-04-30,7.09,7.21,678.48,674.19,674.01
116797,2026-01-01,2020-04-05,7.24,7.36,673.31,669.06,668.87
116798,2026-01-01,2020-05-05,6.91,7.03,685.34,681.00,680.82


In [37]:
df_pre24.describe()

,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
count,112.000000,112.000000,112.000000,112.000000,112.000000
mean,7.871607,7.991607,788.178750,785.423482,785.181786
std,0.508342,0.508342,8.401761,8.345638,8.353013
min,6.410000,6.530000,768.490000,765.740000,765.490000
25%,7.667500,7.787500,782.722500,780.030000,779.780000
50%,8.065000,8.185000,788.725000,786.010000,785.765000
75%,8.230000,8.350000,791.617500,789.017500,788.767500
max,8.450000,8.570000,811.130000,808.060000,807.860000


In [38]:
df_selic.data_vencimento.unique()

array(['2023-01-03T00:00:00.000000000', '2027-01-03T00:00:00.000000000',
       '2025-01-03T00:00:00.000000000', '2024-01-09T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [39]:
df_selic23 = df_selic[df_selic['data_vencimento'] == '2023-01-03']
df_selic24 = df_selic[df_selic['data_vencimento'] == '2024-01-09']
df_selic25 = df_selic[df_selic['data_vencimento'] == '2025-01-03']
df_selic27 = df_selic[df_selic['data_vencimento'] == '2027-01-03']


In [40]:
df_selic24.describe()

,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
count,112.000000,112.000000,112.000000,112.000000,112.000000
mean,0.214107,0.224107,10765.546429,10761.973304,10760.561339
std,0.035043,0.035043,36.574304,36.694376,36.345591
min,0.150000,0.160000,10725.520000,10721.730000,10720.810000
25%,0.180000,0.190000,10737.285000,10733.590000,10732.437500
50%,0.230000,0.240000,10746.720000,10743.080000,10741.835000
75%,0.240000,0.250000,10793.162500,10789.695000,10788.122500
max,0.270000,0.280000,10852.750000,10849.390000,10847.500000


In [41]:
df_ipcajuros.data_vencimento.unique()

array(['2024-08-15T00:00:00.000000000', '2035-05-15T00:00:00.000000000',
       '2030-08-15T00:00:00.000000000', '2040-08-15T00:00:00.000000000',
       '2045-05-15T00:00:00.000000000', '2050-08-15T00:00:00.000000000',
       '2026-08-15T00:00:00.000000000', '2055-05-15T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [42]:
df_ipcajuros24 = df_ipcajuros[df_ipcajuros['data_vencimento'] == '2024-08-15']
df_ipcajuros26 = df_ipcajuros[df_ipcajuros['data_vencimento'] == '2026-08-15']
df_ipcajuros30 = df_ipcajuros[df_ipcajuros['data_vencimento'] == '2030-08-15']
df_ipcajuros35 = df_ipcajuros[df_ipcajuros['data_vencimento'] == '2035-08-15']
df_ipcajuros40 = df_ipcajuros[df_ipcajuros['data_vencimento'] == '2040-08-15']
df_ipcajuros45 = df_ipcajuros[df_ipcajuros['data_vencimento'] == '2045-05-15']
df_ipcajuros50 = df_ipcajuros[df_ipcajuros['data_vencimento'] == '2050-08-15']
df_ipcajuros55 = df_ipcajuros[df_ipcajuros['data_vencimento'] == '2055-05-15']


In [46]:
df_ipcajuros26.head(10)

,data_vencimento,data_base,taxa_compra_manha,taxa_venda_manha,pu_compra_manha,pu_venda_manha,pu_base_manha
94452,2026-08-15,2020-04-08,1.95,2.07,4143.38,4118.46,4117.64
94453,2026-08-15,2020-05-08,1.91,2.03,4152.54,4127.57,4126.75
94454,2026-08-15,2020-06-08,1.97,2.09,4140.84,4115.99,4115.16
94455,2026-08-15,2020-07-08,1.90,2.02,4156.88,4131.92,4130.16
94456,2026-08-15,2020-10-08,1.94,2.06,4149.34,4124.46,4123.64
94457,2026-08-15,2020-11-08,1.99,2.11,4139.75,4114.97,4114.15
94458,2026-08-15,2020-12-08,2.05,2.17,4128.15,4103.47,4102.65
94459,2026-08-15,2020-08-13,2.07,2.19,4124.83,4100.21,4099.38
94460,2026-08-15,2020-08-14,2.18,2.30,4005.53,3981.09,4077.80
94461,2026-08-15,2020-08-17,2.16,2.28,4010.11,3985.66,3985.15


In [47]:
df_ipcajuros26.shape

(1363, 7)

In [ ]:
df_ipcajuros30

In [43]:
df_prejuros.data_vencimento.unique()

array(['2029-01-01T00:00:00.000000000', '2025-01-01T00:00:00.000000000',
       '2027-01-01T00:00:00.000000000', '2023-01-01T00:00:00.000000000',
       '2031-01-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [44]:
df_prejuros23 = df_prejuros[df_prejuros['data_vencimento'] == '2023-01-01']
df_prejuros25 = df_prejuros[df_prejuros['data_vencimento'] == '2025-01-01']
df_prejuros27 = df_prejuros[df_prejuros['data_vencimento'] == '2027-01-01']
df_prejuros29 = df_prejuros[df_prejuros['data_vencimento'] == '2029-01-01']
df_prejuros31 = df_prejuros[df_prejuros['data_vencimento'] == '2031-01-01']
